# Multi-Interval Index Example

This notebook demonstrates how `DimensionInterval` enables automatic cross-slicing between multiple interval types over a shared continuous dimension.

## Use Case: Speech Data

Imagine you have speech data with:
- A **continuous time dimension** (e.g., audio samples at regular intervals)
- **Word intervals** - each word spans a range of time
- **Phoneme intervals** - each phoneme spans a smaller range of time within words

TODO: add an explanatory image


When you select a specific word, you want the time and phoneme dimensions to automatically constrain to only the overlapping values. This is exactly what `DimensionInterval` provides.

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

from linked_indices.multi_interval_index import DimensionInterval

## Creating the Dataset

First, let's create a dataset with:
- 1000 time points from 0 to 120
- 3 words: "run" [0-40), "quickly" [40-80), "home" [80-120)
- 6 phonemes, each spanning 20 time units
- Part of speech labels for each word (verb, adverb, noun)

Note that word intervals are larger and contain multiple phoneme intervals.

In [3]:
C = 2  # number of channels
N = 1000  # number of time points
times = np.linspace(0, 120, N)

# Word intervals - 3 words covering full time range
word_breaks = [0.0, 40.0, 80.0, 120.0]
word_intervals = pd.IntervalIndex.from_breaks(word_breaks, closed="left")
word_labels = ["run", "quickly", "home"]
word_pos = ["verb", "adverb", "noun"]  # part of speech labels

# Phoneme intervals - 6 phonemes, each 20 units
phoneme_breaks = [0.0, 20.0, 40.0, 60.0, 80.0, 100.0, 120.0]
phoneme_intervals = pd.IntervalIndex.from_breaks(phoneme_breaks, closed="left")
phoneme_labels = ["ah", "ee", "oh", "oo", "eh", "ih"]

data = np.random.rand(C, N)

# Create the dataset
ds = xr.Dataset(
    {"data": (("C", "time"), data)},
    coords={
        "time": times,
        "word_intervals": ("word", word_intervals),
        "word": ("word", word_labels),
        "part_of_speech": ("word", word_pos),
        "phoneme_intervals": ("phoneme", phoneme_intervals),
        "phoneme": ("phoneme", phoneme_labels),
    },
)

ds

<xarray.Dataset> Size: 24kB
Dimensions:            (C: 2, time: 1000, word: 3, phoneme: 6)
Coordinates:
  * time               (time) float64 8kB 0.0 0.1201 0.2402 ... 119.9 120.0
  * word               (word) <U7 84B 'run' 'quickly' 'home'
  * phoneme            (phoneme) <U2 48B 'ah' 'ee' 'oh' 'oo' 'eh' 'ih'
    word_intervals     (word) interval[float64, left] 48B [0.0, 40.0) ... [80...
    part_of_speech     (word) <U6 72B 'verb' 'adverb' 'noun'
    phoneme_intervals  (phoneme) interval[float64, left] 96B [0.0, 20.0) ... ...
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 16kB 0.5546 0.5324 ... 0.0192 0.2563

## Applying the Multi-Interval Index

Now we apply `DimensionInterval` to link all these coordinates together. This replaces the default indexes with a single custom index that understands the relationships between time, words, and phonemes.

In [4]:
ds = ds.drop_indexes(["time", "word", "phoneme"]).set_xindex(
    [
        "time",
        "word_intervals",
        "phoneme_intervals",
        "word",
        "part_of_speech",
        "phoneme",
    ],
    DimensionInterval,
)

ds

<xarray.Dataset> Size: 24kB
Dimensions:            (C: 2, time: 1000, word: 3, phoneme: 6)
Coordinates:
  * time               (time) float64 8kB 0.0 0.1201 0.2402 ... 119.9 120.0
  * word               (word) <U7 84B 'run' 'quickly' 'home'
  * phoneme            (phoneme) <U2 48B 'ah' 'ee' 'oh' 'oo' 'eh' 'ih'
  * word_intervals     (word) interval[float64, left] 48B [0.0, 40.0) ... [80...
  * phoneme_intervals  (phoneme) interval[float64, left] 96B [0.0, 20.0) ... ...
  * part_of_speech     (word) <U6 72B 'verb' 'adverb' 'noun'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 16kB 0.5546 0.5324 ... 0.0192 0.2563
Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

Notice in the `Indexes` section that all coordinates are now grouped under a single `DimensionInterval` index. This means selections on any of these coordinates will automatically propagate constraints to the others.

In [8]:
ds.xindexes

Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

The raw data of the intervals are directly encoded in the dataset so we can trivially retrieve them

In [9]:
ds["word_intervals"]

<xarray.DataArray 'word_intervals' (word: 3)> Size: 48B
<IntervalArray>
[[0.0, 40.0), [40.0, 80.0), [80.0, 120.0)]
Length: 3, dtype: interval[float64, left]
Coordinates:
  * word               (word) <U7 84B 'run' 'quickly' 'home'
  * time               (time) float64 8kB 0.0 0.1201 0.2402 ... 119.9 120.0
  * word_intervals     (word) interval[float64, left] 48B [0.0, 40.0) ... [80...
  * phoneme_intervals  (phoneme) interval[float64, left] 96B [0.0, 20.0) ... ...
  * part_of_speech     (word) <U6 72B 'verb' 'adverb' 'noun'
  * phoneme            (phoneme) <U2 48B 'ah' 'ee' 'oh' 'oo' 'eh' 'ih'
Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

It's possible to make multiple selections on metadata at the same time

In [10]:
ds.sel(part_of_speech="adverb", phoneme="oh")

<xarray.Dataset> Size: 4kB
Dimensions:            (C: 2, time: 167, word: 1, phoneme: 1)
Coordinates:
  * time               (time) float64 1kB 40.0 40.12 40.24 ... 59.7 59.82 59.94
  * word               (word) <U7 28B 'quickly'
  * phoneme            (phoneme) <U2 8B 'oh'
  * word_intervals     (word) interval[float64, left] 16B [40.0, 80.0)
  * phoneme_intervals  (phoneme) interval[float64, left] 16B [40.0, 60.0)
  * part_of_speech     (word) <U6 24B 'adverb'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 3kB 0.2229 0.71 0.639 ... 0.4894 0.318
Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

## Cross-Slicing: Selecting on Time

When we select a time range, both word and phoneme dimensions are automatically constrained to only include intervals that overlap with that time range.

In [14]:
# Time 30-70 overlaps:
# - word: [0,40), [40,80) -> 2 words
# - phoneme: [20,40), [40,60), [60,80) -> 3 phonemes
ds.sel(time=slice(30, 70))

<xarray.Dataset> Size: 8kB
Dimensions:            (C: 2, time: 333, word: 2, phoneme: 3)
Coordinates:
  * time               (time) float64 3kB 30.03 30.15 30.27 ... 69.79 69.91
  * word               (word) <U7 56B 'run' 'quickly'
  * phoneme            (phoneme) <U2 24B 'ee' 'oh' 'oo'
  * word_intervals     (word) interval[float64, left] 32B [0.0, 40.0) [40.0, ...
  * phoneme_intervals  (phoneme) interval[float64, left] 48B [20.0, 40.0) ......
  * part_of_speech     (word) <U6 48B 'verb' 'adverb'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 5kB 0.3686 0.5955 ... 0.77 0.4895
Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

## Cross-Slicing: Selecting on Word Interval

Selecting a point within a word interval selects that word and constrains time and phonemes accordingly.

In [15]:
# Selecting at time=60 picks the word interval [40,80) which is "quickly"
# This constrains:
# - time to 40-80
# - phoneme to [40,60), [60,80) -> 2 phonemes
ds.sel(word_intervals=60)

<xarray.Dataset> Size: 8kB
Dimensions:            (C: 2, time: 334, word: 1, phoneme: 2)
Coordinates:
  * time               (time) float64 3kB 40.0 40.12 40.24 ... 79.76 79.88 80.0
  * word               (word) <U7 28B 'quickly'
  * phoneme            (phoneme) <U2 16B 'oh' 'oo'
  * word_intervals     (word) interval[float64, left] 16B [40.0, 80.0)
  * phoneme_intervals  (phoneme) interval[float64, left] 32B [40.0, 60.0) [60...
  * part_of_speech     (word) <U6 24B 'adverb'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 5kB 0.2229 0.71 0.639 ... 0.0536 0.4836
Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

## Cross-Slicing: Selecting on Phoneme Interval

Similarly, selecting a phoneme constrains time and words.

In [16]:
# Selecting at time=70 picks phoneme interval [60,80)
# This constrains:
# - time to 60-80
# - word to [40,80) -> 1 word ("quickly")
ds.sel(phoneme_intervals=70)

<xarray.Dataset> Size: 4kB
Dimensions:            (C: 2, time: 167, word: 1, phoneme: 1)
Coordinates:
  * time               (time) float64 1kB 60.06 60.18 60.3 ... 79.76 79.88 80.0
  * word               (word) <U7 28B 'quickly'
  * phoneme            (phoneme) <U2 8B 'oo'
  * word_intervals     (word) interval[float64, left] 16B [40.0, 80.0)
  * phoneme_intervals  (phoneme) interval[float64, left] 16B [60.0, 80.0)
  * part_of_speech     (word) <U6 24B 'adverb'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 3kB 0.7864 0.7637 ... 0.0536 0.4836
Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

## `isel`

You can isel on any of the dimension coords. In this case `time`, `word`, `phoneme`

This means you can easily perform meaningful selections such as give me the second word that was spoken.

In [17]:
ds.isel(word=1)

<xarray.Dataset> Size: 19kB
Dimensions:            (C: 2, time: 334, word: 1, phoneme: 2)
Coordinates:
  * time               (time) float64 3kB 40.0 40.12 40.24 ... 79.76 79.88 80.0
  * word               (word) <U7 28B 'quickly'
  * phoneme            (phoneme) <U2 16B 'oh' 'oo'
  * word_intervals     (word) interval[float64, left] 16B [40.0, 80.0)
  * phoneme_intervals  (phoneme) interval[float64, left] 32B [40.0, 60.0) [60...
  * part_of_speech     (word) <U6 24B 'adverb'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 16kB 0.5546 0.5324 ... 0.0192 0.2563
Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

or more complex ones such as the 2nd phoneme of the 3rd word

In [21]:
ds.isel(word=2).isel(phoneme=1)

<xarray.Dataset> Size: 17kB
Dimensions:            (C: 2, time: 167, word: 1, phoneme: 1)
Coordinates:
  * time               (time) float64 1kB 100.1 100.2 100.3 ... 119.9 120.0
  * word               (word) <U7 28B 'home'
  * phoneme            (phoneme) <U2 8B 'ih'
  * word_intervals     (word) interval[float64, left] 16B [80.0, 120.0)
  * phoneme_intervals  (phoneme) interval[float64, left] 16B [100.0, 120.0)
  * part_of_speech     (word) <U6 24B 'noun'
Dimensions without coordinates: C
Data variables:
    data               (C, time) float64 16kB 0.5546 0.5324 ... 0.0192 0.2563
Indexes:
  ┌ time               DimensionInterval
  │ word_intervals
  │ phoneme_intervals
  │ word
  │ part_of_speech
  └ phoneme

## Summary

`DimensionInterval` provides:

1. **Automatic cross-slicing** - Select on any dimension and all others are constrained appropriately
2. **Multiple interval types** - Support for multiple interval dimensions (words, phonemes, etc.) over a single continuous dimension
3. **Multiple labels per interval** - Each interval dimension can have multiple label coordinates (e.g., word text and part of speech)
4. **Flexible selection** - Use `sel()` with time values, interval coordinates, or label coordinates

This is particularly useful for:
- Speech/audio data with hierarchical annotations
- Time series with multiple granularities of events
- Any data where intervals at different scales need to stay synchronized